### Compare output of Transformer model and Gru model
- Transformer true | Gru true
- Transformer true | Gru false
- Transformer false | Gru true
- Transformer false | Gru false

In [10]:
def compare_output_trans_gru(out_trans_path, out_gru_path, output_path):
    ftrans = open(out_trans_path, "r", encoding="utf-8")
    fgru = open(out_gru_path, "r", encoding="utf-8")
    t_t = open(output_path + "trans-gru-tt.txt", "w", encoding="utf-8")
    t_f = open(output_path + "trans-gru-tf.txt", "w", encoding="utf-8")
    f_t = open(output_path + "trans-gru-ft.txt", "w", encoding="utf-8")
    f_f = open(output_path + "trans-gru-ff.txt", "w", encoding="utf-8")

    # read title line: s1|s2|prediction|p0|p1|label
    ftrans.readline()
    fgru.readline()

    while (True):
        line_trans = ftrans.readline()
        line_gru = fgru.readline()
        if (line_trans == ""):
            break
        line_trans_split = line_trans.split("\t")
        line_gru_split = line_gru.split("\t")

        trans_pred = line_trans_split[2].strip()
        gru_pred = line_gru_split[2].strip()
        label = line_trans_split[5].strip()

        if (trans_pred == label and gru_pred == label):
            t_t.write(line_trans)
        elif (trans_pred == label and gru_pred != label):
            t_f.write(line_trans)
        elif (trans_pred != label and gru_pred == label):
            f_t.write(line_trans)
        elif (trans_pred != label and gru_pred != label):
            f_f.write(line_trans)
        else:
            print("Error when reading line: ", line_trans)

def f1_score(out_trans_path, output_path):
    ftrans = open(out_trans_path, "r", encoding="utf-8")
    t_t = open(output_path + "trans-tt.txt", "w", encoding="utf-8")
    t_f = open(output_path + "trans-tf.txt", "w", encoding="utf-8")
    f_t = open(output_path + "trans-ft.txt", "w", encoding="utf-8")
    f_f = open(output_path + "trans-ff.txt", "w", encoding="utf-8")

    # read title line: s1|s2|prediction|p0|p1|label
    ftrans.readline()

    while (True):
        line_trans = ftrans.readline()
        if (line_trans == ""):
            break
        line_trans_split = line_trans.split("\t")
        
        pred = line_trans_split[2].strip()
        label = line_trans_split[5].strip()
        
        if (pred == "1" and label == "1"):
            t_t.write(line_trans)
        elif (pred == "1" and label == "0"):
            t_f.write(line_trans)
        elif (pred == "0" and label == "1"):
            f_t.write(line_trans)
        elif (pred == "0" and label == "0"):
            f_f.write(line_trans)
        else:
            print("Error when reading line: ", line_trans)
           

In [11]:
out_trans_path = "./models/finetuned_transformer_ulan_23072022/pred_results_ulan-test.txt"
out_gru_path = "./models/finetuned_gru_ulan_23072022/pred_results_ulan-test.txt"
output_path = "./comparison/ULAN/"
compare_output_trans_gru(out_trans_path, out_gru_path, output_path)
#f1_score(out_trans_path, output_path)

### Count number of "initial only" names that are correctly predicted

In [1]:
def check_initial_name(input_string):
    split = input_string.split(" ")
    for x in split:
        if (len(x) == 2 and x[1] == '.'):
            return True
    return False

def count_true_inital_only(trans_path, gru_path, output_path):
    ftrans = open(trans_path, "r", encoding="utf-8") 
    fgru = open(gru_path, "r", encoding="utf-8") 
    trans_true_gru_false = open(output_path + "initial-trans-true-gru-false.txt", "w", encoding="utf-8")
    
    # read title line: s1|s2|prediction|p0|p1|label
    ftrans.readline()
    fgru.readline()
    
    trans_count, gru_count, sum = 0, 0, 0
    while(True):
        ltrans = ftrans.readline() 
        lgru = fgru.readline() 
        
        if (ltrans == ""):
            break
        line_trans_split = ltrans.split("\t")
        line_gru_split = lgru.split("\t")
    
        s1, s2 = line_trans_split[0], line_trans_split[1]
        trans_predict, gru_predict = line_trans_split[2], line_gru_split[2]
        label = line_trans_split[5].strip()
        
        if ((check_initial_name(s1) or check_initial_name(s2)) and label == "1"):
            sum = sum + 1
            if (trans_predict == label):
                trans_count = trans_count + 1
            if (gru_predict == label):
                gru_count = gru_count + 1

            if (trans_predict == label and gru_predict != label):
                trans_true_gru_false.write(ltrans)
            
    print("- Transformer: ", trans_count, "/", sum, " : ", trans_count/sum) 
    print("- GRU: ", gru_count, "/", sum, " : ", gru_count/sum)        
    

trans_ulan_path = "./models/finetuned_transformer_ulan_23072022/pred_results_ulan-test.txt"
trans_wiki_path = "./models/finetuned_transformer_wikidata_23072022/pred_results_wikidata-test.txt"
gru_ulan_path = "./models/finetuned_gru_ulan_23072022/pred_results_ulan-test.txt"
gru_wiki_path = "./models/finetuned_gru_wikidata_23072022/pred_results_wikidata-test.txt"
output_path = ""

print("Predict \"inital only\" case:")
print("ULAN: ") 
count_true_inital_only(trans_ulan_path, gru_ulan_path, output_path)
print("Wikidata: ") 
#count_true_inital_only(trans_wiki_path, gru_wiki_path, output_path)

Predict "inital only" case:
ULAN: 
- Transformer:  4841 / 5052  :  0.9582343626286619
- GRU:  4716 / 5052  :  0.9334916864608076
Wikidata: 
